# Predicting frauds step by step

# 1 - Introduction

Hi! My name is Yuri and I'm studying data science and machine learning. After taking courses on this subject and practicing with some getting started competitions, I decided to take a look on how a **real** Kaggle competition looks like. In this notebook, my idea is to present my workflow sequentially, without changing the order of the steps that I took, and writing down everything that was important to my decisions. The goal here is not to provide a detailed analysis concisely written, but rather show how my understanding of the problem evolved with data exploration and by reading other notebooks and discussions made by the Kaggle community. I think this notebook will be very useful to me in future projects, and I hope it may be useful to other beginners like me when facing such a challenging competition.

**Finally, I'm still learning from this and other competitions, so, any comments or constructive criticism will be helpful :)**

# 2 - A simple data analysis and modelling

In [ ]:
#Importing libraries.

import os
import gc
import time
import datetime

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn import metrics


#Reading the data.

train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
test_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

First things first! Let's take a look at the training data and discover some important information about it, like the number of columns in the identity and transaction data, the number of frauds, and how much missing information there is in each column.

In [ ]:
print('train_identity has {} columns'.format(len(train_identity.columns)))
train_identity.head()

In [ ]:
print('train_transaction has {} columns'.format(len(train_transaction.columns)))
train_transaction.head()

In [ ]:
print("There are {} fraudulent transactions out of {}".format(train_transaction['isFraud'].sum(),train_transaction.shape[0]))
_ = sns.countplot(x="isFraud", data=train_transaction)

In [ ]:
identity_cols_withNA = 100*train_identity.isnull().sum().sort_values(ascending=False)/train_identity.shape[0]
identity_cols_withNA = identity_cols_withNA[identity_cols_withNA > 0]
print('train_identity has {} columns with null entries. The 10 most empty columns are shown below'.format(identity_cols_withNA.shape[0]))
identity_cols_withNA[:10]

In [ ]:
transaction_cols_withNA = 100*train_transaction.isnull().sum().sort_values(ascending=False)/train_transaction.shape[0]
transaction_cols_withNA = transaction_cols_withNA[transaction_cols_withNA > 0]
print('train_transaction has {} columns with null entries. The 10 most empty columns are shown below'.format(transaction_cols_withNA.shape[0]))
transaction_cols_withNA[:10]

There are many problems already in the beginning. We have a very large dataset, with many missing values, and the columns does not have a clear meaning. The data description only tells us what are the categorical features and states that the column "isFraud" is the target variable, "TransactionDT" is a timedelta, and "TransactionID" is the column that joins the identity and transaction files. By reading [the post made by the competition host](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203) we discover that the field names are purposefully masked for privacy protection and contract agreement. We also are able to understand what are the features related to, but not what many of them actually are. Hence, it is really hard to try things like manually impute the data by using domain knowledge or grasp meaningful information with exploratory data analysis (of course I already took a look at the notebooks that did it, but they did not helped me very much at this point). 

We shall keep it simple then. Let's explore the categorical columns and, to gain further insights over the dataset, also make a simple model without much data cleaning and feature engineering.

In [ ]:
identity_cat_cols = ['DeviceType','DeviceInfo']
for i in range(12,39):
    identity_cat_cols.append('id_'+str(i))

transaction_cat_cols = ['ProductCD','addr1','addr2','P_emaildomain','R_emaildomain']
for i in range(1,7):
    transaction_cat_cols.append('card'+str(i))
    
for i in range(1,10):
    transaction_cat_cols.append('M'+str(i))

In [ ]:
print('Number of unique entries per categorical column (including nulls) in train_identity:\n')
for col in identity_cat_cols:
    cat_number = len(train_identity[col].unique())
    print(col+': '+str(cat_number))

In [ ]:
print('Number of unique entries per categorical column (including nulls) in train_transaction:\n')
for col in transaction_cat_cols:
    cat_number = len(train_transaction[col].unique())
    print(col+': '+str(cat_number))

There are a lot of different categories in some of the features. Card1 has astounding 13553 different categories. DeviceInfo also has over a thousand categories (1787). We must deal with that later.

Before starting modelling, notice that such a large dataset consume a lot of RAM (over 3GB).

In [ ]:
train_identity_mem_usage = train_identity.memory_usage(index = True).sum()/(1024**2)
train_transaction_mem_usage = train_transaction.memory_usage(index = True).sum()/(1024**2)
test_identity_mem_usage = test_identity.memory_usage(index = True).sum()/(1024**2)
test_transaction_mem_usage = test_transaction.memory_usage(index = True).sum()/(1024**2)
print('Memory usage of train_identity is {:.2f} MB'.format(train_identity_mem_usage))
print('Memory usage of train_transaction is {:.2f} MB'.format(train_transaction_mem_usage))
print('Memory usage of test_identity is {:.2f} MB'.format(test_identity_mem_usage))
print('Memory usage of test_transaction is {:.2f} MB'.format(test_transaction_mem_usage))

Unfortunately, I only realized that after trying to run some code that crashed my kernel, but if you are reading this, you won't make the same mistake! There is a way to reduce the memory of a dataset, which consists of converting every categorical column data type to category and the numerical columns to the type that ocuppies the minimum amount of memory possible while also preserving all the information in that column. The code below is inspired in what I found in [other kernel](https://www.kaggle.com/artgor/eda-and-models) and at https://www.mikulskibartosz.name/how-to-reduce-memory-usage-in-pandas/.

In [ ]:
def reduce_mem_usage(df,cat_cols):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in cat_cols:
        df[col] = df[col].astype('category')
        
    num_cols = list(set(df.columns) - set(cat_cols))
    for col in num_cols:
        col_type = df[col].dtype        
        c_min = df[col].min()
        c_max = df[col].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                df[col] = df[col].astype(np.uint8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                df[col] = df[col].astype(np.uint16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                df[col] = df[col].astype(np.uint32)                    
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)
            elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                df[col] = df[col].astype(np.uint64)  
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

It is important to understand what is being done in this function, which can be accomplished by reading the [numpy documentation on its basic data types](https://numpy.org/doc/stable/user/basics.types.html). Now we apply this function to the training data set and merge the identity and transaction dataframes.

In [ ]:
print('Reducing memory usage of train_identity\n')
train_identity = reduce_mem_usage(train_identity,identity_cat_cols)
print('\nReducing memory usage of train_transaction\n')
train_transaction = reduce_mem_usage(train_transaction,transaction_cat_cols)

train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
train_mem_usage = train.memory_usage(index = True).sum()/(1024**2)
print('\nMemory usage of train is {:.2f} MB'.format(train_mem_usage))
train.head()

Now let's do the same with the test set. The test dataset has some columns named slightly different from the training dataset. For convenience, we shall rename them. After merging identity and transaction, it is good to delete the heavy variables and use the gc python module to collect the garbage and free the RAM for further computation.

In [ ]:
cols_to_rename = {}
for i in range(10):    
    cols_to_rename['id-0'+str(i)] = 'id_0'+str(i)
for i in range(10,39):    
    cols_to_rename['id-'+str(i)] = 'id_'+str(i)
    
test_identity = test_identity.rename(columns=cols_to_rename)

print('Reducing memory usage of test_identity\n')
test_identity = reduce_mem_usage(test_identity,identity_cat_cols)
print('\nReducing memory usage of test_transaction\n')
test_transaction = reduce_mem_usage(test_transaction,transaction_cat_cols)

test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')
test_mem_usage = test.memory_usage(index = True).sum()/(1024**2)
print('\nMemory usage of test is {:.2f} MB'.format(test_mem_usage))

del train_transaction
del train_identity
del test_identity
del test_transaction
gc.collect()
test.head()

Now that we have our data in good shape let's start modelling. But first we need to decide how to impute the missing values, right? Well, not necessarily. Gradient boosting models like XGBoost and LightGBM are actually [able to handle missing data](http://mlexplained.com/2018/01/05/lightgbm-and-xgboost-explained/), and since we already discussed that imputing nulls in this dataset is not easy we will not bother with that now. Since [LightGBM is faster than XGBoost](https://towardsdatascience.com/lightgbm-vs-xgboost-which-algorithm-win-the-race-1ff7dd4917d), we will prefer this model here (like the majority of public kernels in this competition). The steps for modelling are the following:


1. **Encode the categorical features -**  This is a tricky part. One-hot-encoding the categorical features allows us to to handle unseen categories, but it is [not optimal for LightGBM](https://lightgbm.readthedocs.io/en/latest/Advanced-Topics.html), which performs better when categories are encoded as integers. LabelEncoder is able to encode categorical data in integers, however, [it is meant to encode the target variable, not the features](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html?highlight=labelencoder#sklearn.preprocessing.LabelEncoder). This means that it only accepts a single column as the input and it cannot handle unseen categories. Since in this competition we only care about the training and test sets, one way to surpass this problem is to use LabelEncoder to encode the categorical features of the training and test set together, as done in https://www.kaggle.com/artgor/eda-and-models.


2. **Search for the best hyperparameters -** this is a necessary step for achieving better scores. However, since the code takes some minutes to run and this is only a simple model made to gain insights over the dataset, [we will not bother with that now](https://www.kaggle.com/c/ieee-fraud-detection/discussion/103109). Instead, we shall use the hyperparameters that were already tuned in other notebooks.


3. **Kfold cross validate the model -** we must do cross validation to see how well our model is able to perform on unseen data that is in the training set and to be able to obtain a better estimation of the feature importances. We evaluate the model using the AUC metric.


4. **Predict the target variable on the test set -** we shall submit our result and see how well our model can perform on unseen data outside the training set. It will be good to compare this score with the mean score on the cross validation sets to draw important conclusions that will guide our future work.

With that in mind, let's start!

In [ ]:
#Step 1

cat_cols = transaction_cat_cols + identity_cat_cols
num_cols = list(set(train.drop(['TransactionID','isFraud','TransactionDT'],axis=1).columns) - set(cat_cols))

train_copy = train.copy()
test_copy = test.copy()
for col in cat_cols:
    le = LabelEncoder()
    le.fit(list(train_copy[col].astype(str).values) + list(test_copy[col].astype(str).values))
    train_copy[col] = le.transform(list(train_copy[col].astype(str).values))
    test_copy[col] = le.transform(list(test_copy[col].astype(str).values))

X = train_copy[cat_cols + num_cols]
y = train_copy['isFraud'] 
X_test = test_copy[cat_cols + num_cols]

del train_copy
del test_copy
gc.collect()

In [ ]:
#Step 2
params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 8,
          'learning_rate': 0.02,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
         }

#Step 3
n_splits = 4
skf = StratifiedKFold(n_splits=n_splits,shuffle=True)
skf.get_n_splits(X, y)

scores = []
i = 1
y_pred = np.zeros(len(X_test.iloc[:,0]))
feat_imp = pd.Series(0, index=X.columns)
for train_index, valid_index in skf.split(X, y):
    print('Training the model using fold number {}...'.format(i))
    X_train, X_valid = X.iloc[train_index,:], X.iloc[valid_index,:]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    model = lgb.LGBMClassifier(**params, n_estimators=500, n_jobs = -1)
    model.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='auc', early_stopping_rounds=200,verbose=False)
    y_pred_valid = model.predict_proba(X_valid)[:, 1]
    y_pred += model.predict_proba(X_test)[:, 1]/n_splits
    score = metrics.roc_auc_score(y_valid, y_pred_valid)
    scores.append(score)
    feat_imp += pd.Series(model.feature_importances_, index=X.columns)/n_splits
    print('AUC score: {:.3f}'.format(score))
    i += 1
    
print('mean AUC score over the validation sets: {:.3f}\n'.format(np.mean(scores)))    
fig,ax = plt.subplots(figsize=(10,10))
_ = feat_imp.nlargest(20).sort_values().plot(kind='barh',ax=ax)
_ = ax.set_xlabel('Average importance (a.u.)')
_ = ax.set_ylabel('Features')

In [ ]:
#Step 4
output = pd.DataFrame({'TransactionID': test.TransactionID, 'isFraud': y_pred})
output.to_csv('simple_lightgbm_submission.csv', index=False)
print("Submission successfully saved!")

**Time to think before taking the next step!** 

The model performance on the test set was about 0.91 (private score, 80% of the data), while the mean score on the cross validation sets is 0.964. This means that our model is probably overfitting the training set. However, it is still surprising that with no work on the data LightGBM was able to achieve this score.

* **Possible reasons for overfitting:** from the feature importance plot, we are able to see that features that are likely to identify the client, such as card1, card2, addr1, dist1, ..., were very important to the model. But what if the test set has unseen clients? Indeed, this is exactly what happens, as explained in the [1st post of the competition's winner](https://www.kaggle.com/c/ieee-fraud-detection/discussion/111284). Another reason is that we used Kfold cross validation, which shuffles the data from different times of the dataset. When applying Kfold cross validation we still use all clients of the training set to train the model, which in turn tend to overfit the training set. Since the clients change over time, a better validation scheme would be to train the model with data in a given interval of time and try to predict frauds in the future. This can be accomplished by using TimeSeriesSplit together with the "TransactionDT" feature. As also discussed in the winner's post, once a client card has fraud, any other future transaction will be caracterized as fraudulent. Hence, we can take advantage of it to post-process the predctions and achieve better results. 

# 3 - Data manipulation and feature engineering

Now it's time to do some data cleaning and manipulation, and some feature engineering to try to improve the model. One important part of the workflow that helps finding good ways of doing this is data visualization. I will not focus into it very much here since there are some great kernels out there with many nice visualizations (like https://www.kaggle.com/ysjf13/cis-fraud-detection-visualize-feature-engineering). Let's start by some basic data cleaning. The first thing that we shall do is to get rid of highly correlated features (most V features are correlated). Chris Deotte, one of the winners of the competition, excluded many V features in a very interesting way, which is presented [in this notebook](https://www.kaggle.com/cdeotte/eda-for-columns-v-and-id). Here I will prefer a rather simple approach, based on computing the correlation matrix and recording the correlated columns (which I found [here](https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/))



In [ ]:
#Cleaning the mess
del X
del X_test
del y
del X_train
del X_valid
del y_train
del y_valid
del y_pred
del y_pred_valid
gc.collect()

In [ ]:
#Sorting the numerical columns by name
num_cols.sort()

# Create correlation matrix
corr_matrix = train[num_cols].corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

print('Showing the first 15 elements of to_drop (out of {}):\n{}'.format(len(to_drop),to_drop[:15]))

As expected, there are many highly correlated V columns. We shall drop all of them (except one for each group of correlated features, of course). There are C and D columns that are also correlated. For those, we shall exclude only the ones with many missing values.

In [ ]:
to_drop_notV = to_drop[:14]
to_drop_V = to_drop[14:]

for col in to_drop_notV:
    na_proportion = train[col].isna().sum()/train.shape[0]
    if na_proportion >= 0.4:
        train = train.drop(col,axis=1)
        test = test.drop(col,axis=1)
        print('Column {} was dropped'.format(col))
        
train = train.drop(columns = to_drop_V)
test = test.drop(columns = to_drop_V)
print('\n{} V columns were dropped'.format(len(to_drop_V)))


Now we deal with the missing values. The idea here is not to impute them with statistics, but just assign them to a value so that LGBM treat them separately (to understand why you must read how LGBM works).

In [ ]:
#Excluding the removed features from the definition of num_cols
num_cols = list(set(num_cols) - set(['D2','D6','D7']) - set(to_drop_V))

train[cat_cols] = train[cat_cols].astype('str').replace('nan','unknown').astype('category')
test[cat_cols] = test[cat_cols].astype('str').replace('nan','unknown').astype('category')
train[num_cols] = train[num_cols].fillna(-999)
test[num_cols] = test[num_cols].fillna(-999)

**Now we shall do some feature engineering**. Feature engineering in this problem seemed very difficult to me, and I took a while reading discussion posts and notebooks trying to understand what techniques could be used in this problem. As already mentioned, we do not have much information about the dataset, which makes things even more difficult. Nonetheless, there are many ways to do feature engineering that does not require a deep understanding of the dataset, just a try and error approach. The many ways to do that are very well explained in [this other topic written by Chris Deotte](https://www.kaggle.com/c/ieee-fraud-detection/discussion/108575), and one good approach to this specific competition can be found in [this notebook by Konstantin Yakovlev](https://www.kaggle.com/kyakovlev/ieee-basic-fe-part-1). Indeed, these guys did an amazing (and hard) work on this problem, and reading their notebooks and discussion posts were by far the moments when I learned the most. A good summary of important posts and notebooks about this competition can be found [here](https://www.kaggle.com/c/ieee-fraud-detection/discussion/107697).

Soo, back to this notebook! To start, let's analyze the categorical features and see what can be done. I analyzed them one by one, trying to find some room for feature engineering. Below I present only the features that I found interesting.

In [ ]:
print('Showing the first 10 unique elements of DeviceInfo (out of {}):\n{}'\
      .format(len(list(train['DeviceInfo'].unique())),\
              list(train['DeviceInfo'].unique())[:10]))

This column has many categories, many of them share some common information, like the name of the device (samsung, motorola etc). One thing that can be done to help the model is to create another column that contains this information (or at least most of them).

In [ ]:
def setDevice(df):
    df['DeviceInfo'] = df['DeviceInfo'].str.lower()   
    df['device_name'] = df['DeviceInfo'].str.split('/', expand=True)[0]
    
    df.loc[df['device_name'].str.contains('sm', na=False), 'device_name'] = 'samsung'
    df.loc[df['device_name'].str.contains('samsung', na=False), 'device_name'] = 'samsung'
    df.loc[df['device_name'].str.contains('gt-', na=False), 'device_name'] = 'samsung'
    df.loc[df['device_name'].str.contains('moto', na=False), 'device_name'] = 'motorola'
    df.loc[df['device_name'].str.contains('lg', na=False), 'device_name'] = 'lg'
    df.loc[df['device_name'].str.contains('rv:', na=False), 'device_name'] = 'rv'
    df.loc[df['device_name'].str.contains('huawei', na=False), 'device_name'] = 'huawei'
    df.loc[df['device_name'].str.contains('ale-', na=False), 'device_name'] = 'huawei'
    df.loc[df['device_name'].str.contains('-l', na=False), 'device_name'] = 'huawei'
    df.loc[df['device_name'].str.contains('blade', na=False), 'device_name'] = 'zte'
    df.loc[df['device_name'].str.contains('linux', na=False), 'device_name'] = 'linux'
    df.loc[df['device_name'].str.contains('xt', na=False), 'device_name'] = 'sony'
    df.loc[df['device_name'].str.contains('htc', na=False), 'device_name'] = 'htc'
    df.loc[df['device_name'].str.contains('asus', na=False), 'device_name'] = 'asus'
    df.loc[df['device_name'].str.contains('lenovo', na=False), 'device_name'] = 'lenovo'
    df.loc[df['device_name'].str.contains('pixel', na=False), 'device_name'] = 'google'
    df.loc[df['device_name'].str.contains('redmi', na=False), 'device_name'] = 'xiaomi'
    df.loc[df['device_name'].str.contains('windows', na=False), 'device_name'] = 'microsoft'
    df.loc[df['device_name'].str.contains('microsoft', na=False), 'device_name'] = 'microsoft'
    df.loc[df['device_name'].str.contains('nexus', na=False), 'device_name'] = 'google'
    df.loc[df['device_name'].str.contains('ilium', na=False), 'device_name'] = 'lanix'
    df.loc[df['device_name'].str.contains('lumia', na=False), 'device_name'] = 'nokia'
    
    df['DeviceInfo'] = df['DeviceInfo'].astype('category')
    df['device_name'] = df['device_name'].astype('category')
    
    return df

In [ ]:
train = setDevice(train)
test = setDevice(test)
print('The new column, device_name, has {} unique values'\
      .format(len(list(train['device_name'].unique()))))

In [ ]:
print('Showing the first 10 unique elements of id_31 (out of {}):\n{}'\
      .format(len(list(train['id_31'].unique())),\
              list(train['id_31'].unique())[:10]))

Same thing could be done here, but since is mixed and there are only 131 categories, I prefer not to create another column.

In [ ]:
train['id_31'] = train['id_31'].str.lower().astype('category')
test['id_31'] = test['id_31'].str.lower().astype('category')

In [ ]:
print('Showing the first 10 unique elements of id_30 (out of {}):\n{}'\
      .format(len(list(train['id_30'].unique())),\
              list(train['id_30'].unique())[:10]))

Here it is easy to get the information about the operating system. Let's do it.

In [ ]:
train['id_30'] = train['id_30'].str.lower()
test['id_30'] = test['id_30'].str.lower()
train['id_30_word1'] = train['id_30'].str.split(' ', expand=True)[0].astype('category')
test['id_30_word1'] = test['id_30'].str.split(' ', expand=True)[0].astype('category')
train['id_30'] = train['id_30'].astype('category')
test['id_30'] = test['id_30'].astype('category')

print('The new column, id_30_word1, has {} unique values'\
      .format(len(list(train['id_30_word1'].unique()))))

In [ ]:
print('Showing the unique elements of id_15:\n{}'\
      .format(list(train['id_15'].unique())))

This column already had an entry called 'Unknown', which has the same meaning as a null value. We must join these two categories.

In [ ]:
train['id_15'] = train['id_15'].str.lower().astype('category')
test['id_15'] = test['id_15'].str.lower().astype('category')

In [ ]:
print('Showing the first 10 unique elements of P_emaildomain (out of {}):\n{}'\
      .format(len(list(train['P_emaildomain'].unique())),\
              list(train['P_emaildomain'].unique())[:10]))

For this column, we can separate the prefix and the suffix to get new information.

In [ ]:
def SplitEmailDomain(df):
    
    df['P_emaildomain_prefix'] = df['P_emaildomain'].str.split('.',1, expand=True)[0].astype('category')
    df['P_emaildomain_sufix'] = df['P_emaildomain'].str.split('.',1, expand=True)[1].fillna('unknown').astype('category')
    df['R_emaildomain_prefix'] = df['R_emaildomain'].str.split('.',1, expand=True)[0].astype('category')
    df['R_emaildomain_sufix'] = df['R_emaildomain'].str.split('.',1, expand=True)[1].fillna('unknown').astype('category')
    
    return df

In [ ]:
train = SplitEmailDomain(train)
test = SplitEmailDomain(test)

train.head()

Now let's move to the D columns. As explained in the data description post, these columns are information abou the number of days since something happened (like the day which the credit card was created). Hence, these features depend not only on the day that 'something happened', but also on the transaction day. To get rid of this mixed information and keep only the information about when something happened, we can subtract the information about the transaction. To do that, it is important to first notice that the column TransactionDT is in seconds and the training set covers 6 months of transactions.

In [ ]:
print('Total number of days:',round((train.TransactionDT.max() - train.TransactionDT.min())/(24*60*60),3))

In [ ]:
# NORMALIZE D COLUMNS
for i in range(1,16):
    if i not in [2,6,7]:
        train['D'+str(i)] = train['D'+str(i)] - train.TransactionDT/np.float32(24*60*60)
        test['D'+str(i)] = test['D'+str(i)] - test.TransactionDT/np.float32(24*60*60)

Now our D features do not contain information about the transaction day.

Ok, we can continue doing feature engineering forever, but it is always a good idea to not do it blindly, but frequently stop and test if the changes we made have a positive or negative impact on our ML model. This is what we will do now. Below, the only difference from what I did in the last section is the cross validation scheme. As explained by Chris Deotte and others, a good way to do cross validation is to train the model on the first few months of transactions (like the first 4 or 5 months) and validate on the last month.

In [ ]:
cat_cols = cat_cols + ['device_name','id_30_word1','P_emaildomain_prefix','P_emaildomain_sufix',\
                      'R_emaildomain_prefix','R_emaildomain_sufix']

train_copy = train.copy()
test_copy = test.copy()
for col in cat_cols:
    le = LabelEncoder()
    le.fit(list(train_copy[col].astype(str).values) + list(test_copy[col].astype(str).values))
    train_copy[col] = le.transform(list(train_copy[col].astype(str).values))
    test_copy[col] = le.transform(list(test_copy[col].astype(str).values))

first_day = train_copy['TransactionDT'].min()
five_months = first_day + 5*30*24*3600
X_train = train_copy.loc[train_copy['TransactionDT'] <= five_months,cat_cols + num_cols]
y_train = train_copy.loc[train_copy['TransactionDT'] <= five_months,'isFraud']
X_valid = train_copy.loc[train_copy['TransactionDT'] > five_months,cat_cols + num_cols]
y_valid = train_copy.loc[train_copy['TransactionDT'] > five_months,'isFraud'] 
X_test = test_copy[cat_cols + num_cols]

del train_copy
del test_copy
gc.collect()

In [ ]:
params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 8,
          'learning_rate': 0.02,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
         }


model = lgb.LGBMClassifier(**params, n_estimators=2000, n_jobs = -1)
model.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='auc', early_stopping_rounds=100,verbose=50)
y_pred_valid = model.predict_proba(X_valid)[:, 1]
y_pred = model.predict_proba(X_test)[:, 1]
score = metrics.roc_auc_score(y_valid, y_pred_valid)
feat_imp = pd.Series(model.feature_importances_, index=X_train.columns)

print('AUC score: {:.3f}'.format(score))
  
fig,ax = plt.subplots(figsize=(10,10))
_ = feat_imp.nlargest(20).sort_values().plot(kind='barh',ax=ax)
_ = ax.set_xlabel('Average importance (a.u.)')
_ = ax.set_ylabel('Features')

In [ ]:
output = pd.DataFrame({'TransactionID': test.TransactionID, 'isFraud': y_pred})
output.to_csv('lightgbm_submission.csv', index=False)
print("Submission successfully saved!")

**Time to think before proceeding again!**

The model did not perform better than before. Actually, it performed a little bit worse. We can also see from the AUC score on the training  and validation set that the model is still overfitting the training set, and it is not very good to generalize to unseen data (unseen clients here). 

We can continue doing feature engineering to try to improve the model, but, for now, I will stop here. The feature engineering that won the competition was already very well explained by Chris Deotte. It consists, basically, on [finding features that uniquely indentify the clients](https://www.kaggle.com/c/ieee-fraud-detection/discussion/111510) (the UIDs), [create aggregated statistics on top of these UIDs](https://www.kaggle.com/c/ieee-fraud-detection/discussion/111453) and then remove the UIDs columns to prevent the model to overfit. To me, the best notebook to follow from this point of the analysis is also one made by Chris Deotte, which can be found [here](https://www.kaggle.com/cdeotte/xgb-fraud-with-magic-0-9600). A lot can be learned by just understanding the meaning of each transformation and the methods used to accomplish that.